In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import LSTM
from tensorflow.keras import backend as K
# from google.colab import files
# from google.colab import drive
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt

In [ ]:
# Create a sequential neural network model
dnn_model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(50),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(50),
    tf.keras.layers.Activation('relu'),
    # tf.keras.layers.Dense(400),
    # tf.keras.layers.Activation('relu'),
    # tf.keras.layers.Dense(200),
    # tf.keras.layers.Activation('relu'),
    # #tf.keras.layers.Dropout(0.4),
    # tf.keras.layers.Dense(20),
    # tf.keras.layers.Activation('relu'),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(4, activation="linear")
])

In [ ]:
#costom loss function
def rmse(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred-y_true)))

def lms(y_true,y_pred):
  return (K.square(y_pred-y_true))/2


In [ ]:
# Compile the model with least mean square loss and Adam optimizer
dnn_model.compile(loss=lms, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)) #metrics=['mse'])

In [ ]:
df = pd.read_csv('J1_Atas Bawah.csv')
df.dropna(inplace=True)

df = df.sample(frac = 1)

X = df[['x', 'y', 'z', 'roll', 'pitch', 'yaw']]
y = df[['theta1', 'theta2', 'theta3', 'theta4']]

train_size = int(0.80 * len(df))
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [ ]:
class myCallback(Callback):    
    def on_epoch_end(self, epoch, logs={}):
        print("Checking loss at end of epoch...")
        if logs['val_loss'] <= 0.00001:
               self.model.stop_training = True
        else:
          print ('not yet...')

In [ ]:
# Train the model on the dataset
# epochs = 200
# batch_size = 20
# dnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=1, batch_size=batch_size)
loss_callback_obj = myCallback()
dnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10000, batch_size=200, verbose=1, callbacks=[loss_callback_obj])

In [ ]:
history_df =pd.DataFrame(dnn_model.history.history)
loss_plot = history_df[['loss', 'val_loss']].plot()
fig = loss_plot.get_figure()
fig.savefig('E:/ACA/File ITS/Jurusan/Kuliah Akademik/kuliah semester 8/Tugas Akhir/Dokumentasi/Grafik/Grafik Error PO/e1_h2_n50_n50 (e0.00001).png')
#fig.savefig('E:/ACA/File ITS/Jurusan/Kuliah Akademik/kuliah semester 8/Tugas Akhir/Dokumentasi/Grafik/Grafik Error PO/e1_h5_n50_n500_n400_n200_n20 (lr0.1 e0.00001).png')

In [ ]:
X_target = X_test.iloc[0:].values
Y_target = y_test.iloc[0:].values

Y_prediksi = dnn_model.predict(X_target)

In [ ]:
asd = np.concatenate([X_target, Y_target, Y_prediksi], axis=1)
column_labels = ['x','y','z','roll','pitch','yaw','theta1','theta2','theta3','theta4','theta1_Pred','theta2_pred','theta3_pred','theta4_pred']
df = pd.DataFrame(asd, columns=column_labels)
df

In [ ]:
dataf = pd.DataFrame({'theta1':Y_target[:,0], 'theta1_pred':Y_prediksi[:,0],
                  'theta2':Y_target[:,1], 'theta2_pred':Y_prediksi[:,1],
                  'theta3':Y_target[:,2], 'theta3_pred':Y_prediksi[:,2],
                  'theta4':Y_target[:,3], 'theta4_pred':Y_prediksi[:,3]}).sort_values(by=['theta1'])

fig, ax = plt.subplots(nrows=2, ncols=2,figsize=(10,5))
axes = ax.flatten()

axes[0].plot([i for i in range(len(dataf))], dataf['theta1'], label='theta1')
axes[0].plot([i for i in range(len(dataf))], dataf['theta1_pred'], label='theta1_pred')
axes[0].set_title('theta1')
axes[0].legend(bbox_to_anchor=(1.1, 1.05))

axes[1].plot([i for i in range(len(dataf))], dataf['theta2'], label='theta2')
axes[1].plot([i for i in range(len(dataf))], dataf['theta2_pred'], label='theta2_pred')
axes[1].set_title('theta2')
axes[1].legend(bbox_to_anchor=(1.1, 1.05))

axes[2].plot([i for i in range(len(dataf))], dataf['theta3'], label='theta3')
axes[2].plot([i for i in range(len(dataf))], dataf['theta3_pred'], label='theta3_pred')
axes[2].set_title('theta3')
axes[2].legend(bbox_to_anchor=(1.1, 1.05))

axes[3].plot([i for i in range(len(dataf))], dataf['theta4'], label='theta4')
axes[3].plot([i for i in range(len(dataf))], dataf['theta4_pred'], label='theta4_pred')
axes[3].set_title('theta4')
axes[3].legend(bbox_to_anchor=(1.1, 1.05))

plt.tight_layout()
dataf
plt.savefig('E:/ACA/File ITS/Jurusan/Kuliah Akademik/kuliah semester 8/Tugas Akhir/Dokumentasi/Grafik/Grafik Joint PO/J1_h2_n50_n50 (e0.00001).png')
#plt.savefig('E:/ACA/File ITS/Jurusan/Kuliah Akademik/kuliah semester 8/Tugas Akhir/Dokumentasi/Grafik/Grafik Joint PO/J1_h5_n50_n500_n400_n200_n20(lr0.1 e0.00001).png')

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(Y_target, Y_prediksi)

In [ ]:
rmse(Y_target, Y_prediksi)

In [ ]:
df.to_csv('E:/ACA/File ITS/Jurusan/Kuliah Akademik/kuliah semester 8/Tugas Akhir/Data/Data Hidden dan Neuron/posisi orientasi/test_h2_n50_n50 (e0.00001).csv', index=False)
#df.to_csv('E:/ACA/File ITS/Jurusan/Kuliah Akademik/kuliah semester 8/Tugas Akhir/Data/Data Hidden dan Neuron/posisi orientasi/test_h5_n50_n500_n400_n200_n20(lr0.1 0.00001).csv', index=False)

In [ ]:
# X_target = X_train.iloc[0:].values
# Y_target = y_train.iloc[0:].values

# Y_prediksi = dnn_model.predict(X_target)

# asd = np.concatenate([Y_target, Y_prediksi], axis=1)
# column_labels = ['theta1','theta2','theta3','theta4','P_theta1','P_theta2','P_theta3','P_theta4'];
# df = pd.DataFrame(asd, columns=column_labels)
# df.plot();

In [ ]:
dnn_model.summary()